## 1. Load trajectory data

In [5]:
import pandas as pd
import os

data_path = "trajectory.csv"
traj = pd.read_csv(data_path)

coords = []
for i in range(len(traj)):
    if traj.Visibility[i] == 1:
        coords.append([traj.X[i], traj.Y[i]])
    else:
        coords.append(None)
coords


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [610, 353],
 [611, 328],
 [611, 328],
 [618, 312],
 [621, 298],
 [623, 285],
 [626, 276],
 [628, 266],
 [628, 266],
 [631, 260],
 [633, 256],
 [636, 252],
 [637, 251],
 [638, 248],
 None,
 None,
 None,
 [651, 255],
 [655, 255],
 [657, 256],
 [657, 258],
 [661, 258],
 [665, 265],
 [671, 271],
 [673, 277],
 [676, 283],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [701, 376],
 [703, 390],
 [696, 378],
 [695, 382],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [628, 262],
 [626, 261],
 [621, 256],
 [621, 256],
 [616, 256],
 [612, 253],
 [610, 256],
 [606, 256],
 [603, 260],
 [602, 260],
 [601, 263],
 [606, 266],
 [630, 266],
 None,
 None,
 None,
 None,
 None,
 [701, 271],
 [712, 276],
 [726, 281],
 [726, 281],
 [736, 288],
 [748, 296],
 [758, 306],
 [770, 316],
 [780, 326],
 [780, 326],
 [788, 338],
 [798, 351],
 [806, 363],
 [811, 365],
 [800, 321],
 [798, 322],
 [793,

## 2. Visualize shuttlecock

In [6]:
import cv2


def plot_shuttlecock(
    coords,
    video_path="input.mp4",
    output_path="output.mp4",
):
    cap = cv2.VideoCapture(video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (1280, 720))

    for i in range(len(coords)):
        ret, frame = cap.read()
        if not ret:
            break

        if coords[i] is not None:
            cv2.circle(
                frame, (int(coords[i][0]), int(coords[i][1])), 5, (0, 0, 255), -1
            )

        out.write(frame)

    cap.release()
    out.release()


plot_shuttlecock(coords)


## 3. Remove outlier data

In [ ]:
import numpy as np

diff_list = []
for i in range(len(coords) - 1):
    if coords[i] is not None and coords[i + 1] is not None:
        point1 = coords[i]
        point2 = coords[i + 1]
        diff = [abs(point2[0] - point1[0]), abs(point2[1] - point1[1])]
        diff_list.append(diff)
    else:
        diff_list.append(None)

x = np.array([x[0] if x is not None else np.nan for x in diff_list])
y = np.array([x[1] if x is not None else np.nan for x in diff_list])

ids = set(np.where(x > 50)[0]) & set(np.where(y > 50)[0])
for id in ids:
    left, middle, right = coords[id - 1], coords[id], coords[id + 1]
    if left is None:
        left = [0]
    if right is None:
        right = [0]
    if middle is None:
        middle = [0]
    MAX = max(map(list, (left, middle, right)))
    if MAX == [0]:
        pass
    else:
        try:
            coords[coords.index(tuple(MAX))] = None
        except ValueError:
            coords[coords.index(MAX)] = None

plot_shuttlecock(coords, output_path="output2.mp4")

## 4. Interpolate missing spots

In [ ]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.
    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """
    return np.isnan(y), lambda z: z.nonzero()[0]

x, y = [x[0] if x is not None else np.nan for x in coords], [x[1] if x is not None else np.nan for x in coords]

xxx = np.array(x) # x coords
yyy = np.array(y) # y coords

nons, yy = nan_helper(xxx)
xxx[nons]= np.interp(yy(nons), yy(~nons), xxx[~nons])
nans, xx = nan_helper(yyy)
yyy[nans]= np.interp(xx(nans), xx(~nans), yyy[~nans])

coords = [*zip(xxx,yyy)]

plot_shuttlecock(coords, output_path="output3.mp4")